In [ ]:
import pandas as pd
data = pd.read_csv('apple_stock_data.csv')
data.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Date       252 non-null    datetime64[ns, UTC]
 1   Adj Close  252 non-null    float64            
 2   Close      252 non-null    float64            
 3   High       252 non-null    float64            
 4   Low        252 non-null    float64            
 5   Open       252 non-null    float64            
 6   Volume     252 non-null    int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1)
memory usage: 13.9 KB


In [ ]:
data.set_index('Date', inplace=True)
data.head()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [ ]:
data = data[['Close']]

In [ ]:
print(data)

                                Close
Date                                 
2023-11-02 00:00:00+00:00  177.570007
2023-11-03 00:00:00+00:00  176.649994
2023-11-06 00:00:00+00:00  179.229996
2023-11-07 00:00:00+00:00  181.820007
2023-11-08 00:00:00+00:00  182.889999
...                               ...
2024-10-28 00:00:00+00:00  233.399994
2024-10-29 00:00:00+00:00  233.669998
2024-10-30 00:00:00+00:00  230.100006
2024-10-31 00:00:00+00:00  225.910004
2024-11-01 00:00:00+00:00  222.910004

[252 rows x 1 columns]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

<ipython-input-9-24eff1bed47e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Close'] = scaler.fit_transform(data[['Close']])


In [ ]:
import numpy as np
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [ ]:
seq_length = 60
X, y = create_sequences(data['Close'].values, seq_length)

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 0.2568
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0389
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0422
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0176
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0216
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0148
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0139
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0118
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0107
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0100
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0101
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0100
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0094
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0088
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0087
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━

In [ ]:
data['lag_1'] = data['Close'].shift(1)
data['lag_2'] = data['Close'].shift(2)
data['lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [ ]:
X_lin = data[['lag_1', 'lag_2', 'lag_3']]
y_lin = data['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [ ]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)

LinearRegression()

In [ ]:
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 339ms/step


In [ ]:
lin_predictions = lin_model.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

In [ ]:
# Assuming train_size is calculated based on the length of the LSTM data (X)
train_size_lstm = int(len(X) * 0.8)  # Or however it was originally calculated

# Calculate a new train_size for the linear regression data based on its length
train_size_lin = int(len(X_lin) * 0.8)

# Split data for linear regression using the new train_size_lin
X_train_lin, X_test_lin = X_lin[:train_size_lin], X_lin[train_size_lin:]
y_train_lin, y_test_lin = y_lin[:train_size_lin], y_lin[train_size_lin:]

# ... (Rest of your code for training and prediction) ...

# Before calculating hybrid_predictions, ensure both arrays have the same number of rows
min_size = min(lstm_predictions.shape[0], lin_predictions.shape[0])

# Use slicing to select the same number of rows from both arrays
hybrid_predictions = (0.7 * lstm_predictions[:min_size]) + (0.3 * lin_predictions[:min_size])

In [ ]:
hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)

ValueError: operands could not be broadcast together with shapes (39,1) (96,1) 